# 取得台股日線資料

到台灣證券交易所取得資料
[證交所](https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20201119&type=ALLBUT0999)

In [9]:
import pandas as pd
import requests
import io

In [19]:
url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20201119&type=ALLBUT0999'
page = requests.get(url)

In [12]:
pd.read_csv(io.StringIO('1,2,3 \n 1,2,3'))

,1,2,3
0,1,2,3


## 1. 將文字檔處理成pandas可以處理的格式

In [97]:
use_text = page.text.splitlines()
use_text[205] ## 作弊直接從excel看他在第幾行

'"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",'

### 目標是抓從證券代號開始的表格 所以搜尋他是在第幾行

* 作弊直接從excel看
* 用迴圈搜尋

In [85]:
for i, data in enumerate([2,4,6]):
    print(i, data)

0 2
1 4
2 6


In [75]:
for i, text in enumerate(use_text):
    if text.startswith('"證券代號","證券名稱","成交股數","成交筆數"') == 1:
        initial_point = i
        break

In [128]:
joinstr = ''.join([text[:-1] + '\n' for text in use_text[initial_point:]]) 
## 取代掉每個元素結尾的逗號換成換行符號 在把所有字串連接在一起
test_df = pd.read_csv(io.StringIO(joinstr))
test_df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,"=""0050""",元大台灣50,"4,505,205","3,786","514,299,847",114.45,114.55,113.80,114.15,-,0.20,114.10,18,114.15,109,0.00
1,"=""0051""",元大中型100,"54,075",42,"2,237,445",41.26,41.45,41.26,41.42,+,0.16,41.40,2,41.41,2,0.00
2,"=""0052""",富邦科技,"1,106,750",342,"111,937,331",102.00,102.00,100.90,101.05,-,0.65,101.00,12,101.05,9,0.00
3,"=""0053""",元大電子,"34,197",19,"1,846,611",54.20,54.30,53.95,54.05,-,0.50,54.05,1,54.40,1,0.00
4,"=""0054""",元大台商50,"10,000",7,"267,310",26.70,26.80,26.70,26.80,+,0.10,26.78,50,26.79,1,0.00


In [130]:
test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"', ''))
test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('=', ''))
test_df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"4,505,205","3,786","514,299,847",114.45,114.55,113.80,114.15,-,0.20,114.10,18,114.15,109,0.00
1,0051,元大中型100,"54,075",42,"2,237,445",41.26,41.45,41.26,41.42,+,0.16,41.40,2,41.41,2,0.00
2,0052,富邦科技,"1,106,750",342,"111,937,331",102.00,102.00,100.90,101.05,-,0.65,101.00,12,101.05,9,0.00
3,0053,元大電子,"34,197",19,"1,846,611",54.20,54.30,53.95,54.05,-,0.50,54.05,1,54.40,1,0.00
4,0054,元大台商50,"10,000",7,"267,310",26.70,26.80,26.70,26.80,+,0.10,26.78,50,26.79,1,0.00


## 2.將以上內容包裝成一個function 輸入多個日期回傳多個df

In [131]:
def crawler(data_time):
    page_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + data_time + "&type=ALLBUT0999"
    page = requests.get(page_url)
    use_text = page.text.splitlines()
for i, text in enumerate(use_text):
    if text.startswith('"證券代號","證券名稱","成交股數","成交筆數"') == 1:
        initial_point = i
        break
    joinstr = ''.join([text[:-1] + '\n' for text in use_text[initial_point:]]) 
    ## 取代掉每個元素結尾的逗號換成換行符號 在把所有字串連接在一起
    test_df = pd.read_csv(io.StringIO(joinstr))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"', ''))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('=', ''))
    return test_d
            

In [137]:
crawler('20201119').head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"4,505,205","3,786","514,299,847",114.45,114.55,113.80,114.15,-,0.20,114.10,18,114.15,109,0.00
1,0051,元大中型100,"54,075",42,"2,237,445",41.26,41.45,41.26,41.42,+,0.16,41.40,2,41.41,2,0.00
2,0052,富邦科技,"1,106,750",342,"111,937,331",102.00,102.00,100.90,101.05,-,0.65,101.00,12,101.05,9,0.00
3,0053,元大電子,"34,197",19,"1,846,611",54.20,54.30,53.95,54.05,-,0.50,54.05,1,54.40,1,0.00
4,0054,元大台商50,"10,000",7,"267,310",26.70,26.80,26.70,26.80,+,0.10,26.78,50,26.79,1,0.00


## 3. 利用 datatime 物件操作加一天減一天

In [139]:
import datetime
import time

In [153]:
date = datetime.datetime.now()
date

datetime.datetime(2020, 11, 19, 23, 37, 21, 388353)

In [154]:
date = str(date).split(' ')[0]
date

'2020-11-19'

In [155]:
date = ''.join(date.split('-'))
date

'20201119'

In [156]:
def trans_date(date_time):
    return ''.join(str(date_time).split(' ')[0].split('-'))

In [164]:
def parse_n_days(start_date, n):
    df_dict = {}
    now_date = start_date
    for i in range(n):
        time.sleep(3)
        now_date = now_date - datetime.timedelta(days = 1)
        try:
            df = crawler(trans_date(now_date))
            print("Success!" + ' ' + trans_date(now_date))
            df_dict.update({trans_date(now_date):df})
        except:
            print('Fails at' + ' ' + trans_date(now_date))
    return df_dict           

In [165]:
result_dict = parse_n_days(datetime.datetime.now(), 6)

Success! 20201118
Success! 20201117
Success! 20201116
Fails at 20201115
Fails at 20201114
Success! 20201113


In [166]:
result_dict.keys()

dict_keys(['20201118', '20201117', '20201116', '20201113'])